In [17]:
%%writefile going_modular/data_setup.py

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = 1

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size:int,
                       num_workers:int=NUM_WORKERS):
    """Create Dataloader for train and test dataset

    Args:
        train_dir (str): path to train dir
        test_dir (str): path to test dir
        transform (transforms.Compose): tranform the 
        batch_size (int): no of items per batch
        num_workers (int, optional):  Defaults to NUM_WORKERS.

    Returns:
        train dataloader and test_dataloader
    """
    train_data = datasets.ImageFolder(root=train_dir,
                                      transform=transform,
                                      )
    
    test_data = datasets.ImageFolder(root=test_dir,
                                      transform=transform,
                                      )
    
    train_dataloader = DataLoader(dataset=train_data, shuffle=True,
                                  batch_size=batch_size,
                                  pin_memory=True)
    
    test_dataloader = DataLoader(dataset=test_data, shuffle=False,
                                 batch_size=batch_size, pin_memory=True)
    
    class_names = train_data.classes
    
    return train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_setup.py


In [19]:
# import thr python script to notebook
from going_modular import data_setup



In [20]:
%%writefile going_modular/model_builder.py
# create model builder

# Build the model
from torch import nn

class TinyVGG(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()
         
        self.conv_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1,padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )
        
        self.conv_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1,padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            
        )
        
        self.classifer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units *13*13, 
                      out_features=output_shape)
            
        )
    
    def forward(self, x):
        x = self.conv_1(x)
        # print(f"After 1st conv : {x.shape}")
        x = self.conv_2(x)
        # print(f"After 2nd conv : {x.shape}")
        x = self.classifer(x)
        # print(f"After Classifier : {x.shape}")
        return x

Writing going_modular/model_builder.py


In [24]:
from going_modular import model_builder
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_builder.TinyVGG(3,10,3).to(device)

TinyVGG(
  (conv_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [25]:
%%writefile going_modular/engine.py
# lets create train and test loop engine

from torch import nn
# create the train setup
from torchmetrics import Accuracy


def train_step(model:nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn : nn.Module,
               optimizer:torch.optim.Optimizer,
               num_class = 3,
               device = device):
    
    accuracy = Accuracy(task="multiclass", num_classes=num_class)

    
    # 1. put the model in train model
    model.train()
    # set metric
    train_loss, train_acc = 0, 0
    
    #loop on dataloader
    for batch, (X,y) in enumerate(data_loader):
        #send the data to target device
        X , y = X.to(device),  y.to(device)
        # forward pass
        train_logits = model_0(X)
        # calc the loss
        loss = loss_fn(train_logits, y)
        train_loss += loss
        # optimizer zero grad
        optimizer.zero_grad()
        #back pass
        loss.backward()
        # adjust the optimizer
        optimizer.step()
        
        # calc accuracy
        
        y_pred_class = torch.argmax(torch.softmax(train_logits, dim=1), dim=1)
        train_acc += accuracy(y_pred_class, y)
    
    
    train_acc = train_acc / len(data_loader)
    train_loss = train_loss / len(data_loader)
    
    return train_loss, train_acc
        
    

def test_step(model:nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn : nn.Module,
              num_class = 3,
              device = device):
    
    accuracy = Accuracy(task="multiclass", num_classes=num_class)

    
    model.eval()
    # since its a dataloader, set test_loss and acc
    test_loss, test_acc = 0, 0
    
    # turn thr infereance mode
    with torch.inference_mode():
    
        for X,y in data_loader:
            
            # send data to target device
            X, y = X.to(device), y.to(device)
            
            #forward pass
            y_preds = model(X)
            
            loss = loss_fn(y_preds, y)
            
            test_loss += loss
            
            y_pred_label = torch.argmax(y_preds , dim=1)
            
            test_acc += accuracy(y_pred_label, y)
            
    test_loss = test_loss /len(data_loader)
    test_acc = test_acc / len(data_loader)
    
    return test_loss, test_acc
         
    
    
    
    
    



Writing going_modular/engine.py
